# METODOS NUMERICOS

## Preambulo

### Librerias

In [4]:
import math as M # constantes matematicas
#import cmath as Cm # constantes matematicas complejas
import numpy as np # arrays y funciones matematicas
import time  # libreria para medir el tiempo de los metodos
from tqdm import tqdm # libreria que nos permite colocar barras de carga de los procesos
import pandas as pd #libreria para generar las tablas

# NOTA: VERIFICAR QUE TODAS LAS LIBRERIAS ESTEN INSTALADAS EN SU COMPUTADOR: TQDM POR EJEMPLO

In [5]:
# inicio = time.time()
# fin = time.time()
# print(fin - inicio)

### Datos iniciales

In [6]:
#Raices de la funcion func1 y func2
root1 = 1+0.0*1j
root2 = np.exp((2/3)*M.pi*1j)
root3 = np.exp((4/3)*M.pi*1j) # UTilizamos esta raiz para R_s
roots = np.array([root1, root2, root3])
# Datos iniciales y tolerancias
tol = 1e-8
maxite = 40

### Funciones auxiliares

In [28]:
# Simplificamos la expresion de las funciones que usaremos mas adelante
def exp(z):
    x = z.real
    y = z.imag
    return (M.expm1(x)+1)*(M.cos(y) + M.sin(y)*1j)
# La funcion M.expm1() solo toma variables punto flotante y mas estable que np.exp.() en valores cercanos al cero
def sin(z):
    return np.sin(z)

def cos(z):
    return np.cos(z)

def matrix_elem_sum(matrix):
    """
    Retorna la suma total de las entradas de la matriz
    """
    matrix = matrix.flatten()
    sum_ = 0
    for entry in list(matrix):
        sum_ += entry
    return sum_
def count_not_conv_points(matrix,roots):
    """
    Retorna el numero de entradas de la matriz que no son iguales a
    ninguna de la raices del arreglo roots.
    """
    count = 0
    matrix = matrix.flatten()
    for entry in list(matrix):
        if (abs(entry*np.ones(roots.size)-roots) > 1e-4).all():
            count += 1
    return count

### Funciones y derivadas

In [8]:
# Lista de funciones y sus derivadas
# Todas estas funciones son escalares y no son funciones vectorizadas
# Al vectorizar las funciones los tiempos de calculo son mayores en estos casos
def func1(z):
    return z**3 - 1

def dfunc1(z):
    return 3 * z**2

def d2func1(z):
    return 6 * z

def func2(z):
    r = 0.01
    return exp(sin(z)*r)*func1(z)

def dfunc2(z):
    r = 0.01
    return exp(sin(z)*r)*(r*cos(z)*func1(z) + dfunc1(z))

def d2func2(z):
    return exp(sin(z)*0.01)*(((0.01*cos(z))**2 - 0.01*sin(z))*func1(z) + 0.02 * cos(z) * dfunc1(z) + d2func1(z))
## Para simplificar el codigo se opta por utilizar un array con todas las derivadas que el metodo necesite de la funcion
def Dfunc1(z):
    return [dfunc1(z),d2func1(z)]

def Dfunc2(z):
    return [dfunc2(z),d2func2(z)]

## IMPLEMENTACION DE LOS METODOS

In [9]:
def Newton_classicroot(func, dfunc, x0, root, tol, maxite):
    """
    Implementación del método de Newton para encontrar las raíces de una función en números complejos.

    Parámetros:
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - Las iteraciones del metodo usando x0 y la raiz encontrada x
    """
    x = x0
    error = 1
    ite = 0
    while (error > tol) and (ite < maxite):
        try:
            x = x - func(x)/dfunc(x)[0]
            error = abs(func(x)) # abs(func(x)) abs(root - x)
            ite += 1
        except Exception:
            ite = maxite # Indicamos que no hay convergencia a partir del x0
            break
    return ite,x

In [29]:
def Newton_classicroot(func, dfunc, x0, root, tol, maxite):
    """
    Implementación del método de Newton para encontrar las raíces de una función en números complejos.

    Parámetros:
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - Las iteraciones del metodo usando x0 y la raiz encontrada x
    """
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(root - x) < tol: # abs(func(x)) abs(root - x)
                return ite, x
            x = x - func(x)/dfunc(x)[0]
        except Exception:
            ite = maxite # Indicamos que no hay convergencia a partir del x0
            break
    return ite, x

### Newton clasico

### Newton raices multiples

In [11]:
def Newton_multipleroot(func,dfunc,x0,root,tol,maxite):
    """
    Implementación del método de Newton para encontrar las raíces complejas multiples.

    Parámetros:
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - Las iteraciones del metodo usando x0 y la raiz encontrada x
    """
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root - x)
                return ite, x
            x = x - (func(x)*dfunc(x)[0])/((dfunc(x)[0])**2 - func(x)*dfunc(x)[1])
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

### Aceleracion convexa de Wittaker

In [12]:
def Convex_acc_Wh(func,dfunc,x0,root,tol,maxite):
    """
    Implementación del método de Newton para encontrar las raíces complejas multiples.

    Parámetros:
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - Las iteraciones del metodo usando x0 y la raiz encontrada x
    """
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            d2fx = dfunc(x)[1]
            L = (fx * d2fx) / (dfx)**2
            x = x - (fx/(2*dfx))*(2-L)
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

### Aceleracion doble de Wittaker

In [13]:
def Double_convex_acc_Wh(func,dfunc,x0,root,tol,maxite):
    """
    Implementación del método de Newton para encontrar las raíces complejas multiples.

    Parámetros:
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - Las iteraciones del metodo usando x0 y la raiz encontrada x
    """
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            d2fx = dfunc(x)[1]
            L = (fx * d2fx) / (dfx)**2
            x = x - (fx/(4*dfx))*(2-L + (4+2*L)/(2-L*(2-L)))
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [14]:
def Halley(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            d2fx = dfunc(x)[1]
            L = (fx * d2fx) / (dfx)**2
            x = x - (fx/dfx) * (2/(2 - L))
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [15]:
def Chebyshev(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            d2fx = dfunc(x)[1]
            L = (fx * d2fx) / (dfx)**2
            x = x - (fx/dfx) * (1 + L/2)
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [16]:
def ConvexN_sHalley(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            d2fx = dfunc(x)[1]
            L = (fx * d2fx) / (dfx)**2
            x = x - (fx/(2*dfx)) * (2 - L)/(1 - L)
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [17]:
def Stir(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            x = x - func(x)/dfunc(x-func(x))[0]
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [18]:
def Steffensen(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            gx = (func(x + func(x)) - func(x))/func(x)
            x = x - fx/gx
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [19]:
def Midpoint_Mth(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            x = x - fx/dfunc(x - fx/(2*dfx))[0]
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [20]:
def Traub_Ostro(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            ux = fx/dfx
            x = x - ux*((func(x-ux)-fx)/(2*func(x-ux)-fx))
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [21]:
def Jarrat(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            ux = fx/dfx
            x = x - 0.5*ux + fx/(dfx - 3*dfunc(x-(2/3)*ux)[0])
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

In [22]:
def Inverse_Jarrat(func,dfunc,x0,root,tol,maxite):
    x = x0
    for ite in range(maxite+1):
        try:
            if abs(func(x)) < tol: # abs(func(x)) abs(root -x)
                return ite, x
            fx = func(x)
            dfx = dfunc(x)[0]
            ux = fx/dfx
            hx = (dfunc(x-(2/3)*ux)[0] - dfx)/dfx
            x = x - ux + 0.75*ux*hx*(1 - 1.5*hx)
        except Exception:
                ite = maxite # Indicamos que no hay convergencia a partir del x0
                break
    return ite, x

## OBTENCION DE LAS TABLAS

### Funciones auxiliares para generar tablas

In [23]:
def parameters_values_method(method,Domfx,Domfy,func,dfunc,roots,tol,maxite):
    """
    Implementación de una funcion auxiliar para generar los parametros numericos restantes de las tablas
    del paper.

    Parámetros:
    - method: metodo de la cual queremos obtener sus parametros.
    - func: la función dada.
    - dfunc: las derivadas de la función f.
    - roots: arreglo de las raices de la funcion f
    - x0: valor inicial para comenzar la iteración.
    - tol: tolerancia para la convergencia.
    - maxite: número máximo de iteraciones permitidas.

    Retorna:
    - NC: porcentaje de total puntos evaluados para los cuales el metodo no converge.
    - IP: iteraciones totales/ total de puntos.
    - T: tiempo de ejecucion del metodo.
    - PS: puntos totales/ tiempo de ejecucion.
    - IS: iteraciones totales / tiempo de ejecucion.
    - x: matrix de los puntos de convergencia del metodo sobre cada elemento en Domf.
    - iterations: matrix de las iteraciones del metodo sobre cada elemento en Domf.
    """
    old_settings = np.seterr(all='ignore') # Desactivamos todas las alertas de la libreria Numpy
    N = Domfx.size
    total_points = N**2
    iterations = []
    x = []
    with tqdm(total=total_points) as pbar:
        start = time.time()
        for i in list(Domfx):
            for j in list(Domfy):
                ite, xx = method(func,dfunc,complex(i,j),roots[1],tol,maxite)
                iterations.append(ite)
                x.append(xx)
                pbar.update(1)  # Actualizar la barra de progreso
        end = time.time()
    x = np.array(x)
    iterations = np.array(iterations)
    total_iterations = matrix_elem_sum(iterations)
    T = end - start
    NC = (count_not_conv_points(x,roots))/total_points
    IP = total_iterations/total_points
    PS = total_points/T
    IS = total_iterations/T
    np.seterr(**old_settings)
    return np.array([NC,IP,T,PS,IS])

## Tablas R_s y R_b

In [24]:
#Numero de puntos
Points = 1024 # Numero de prueba, pero para el computo final seria con 1024
#Creacion de R_s
xRs = np.linspace(-0.6,-0.4,Points)
yRs = np.linspace(0.75,0.95,Points)
#xxRs, yyRs = np.meshgrid(xRs,yRs) #Mallado de Rs
#Creacion de R_b
xRb = np.linspace(-2.5,2.5, Points)
#xxRb, yyRb = np.meshgrid(xRb,xRb) #Mallado de Rb

#Dominios Rs y Rb
#Rs = xxRs + yyRs*1j
#Rb = xxRb + yyRb*1j

#Metodos implementados
Methods = np.array((Newton_multipleroot,Convex_acc_Wh,Double_convex_acc_Wh,Halley,Chebyshev,ConvexN_sHalley,Stir,Steffensen,Midpoint_Mth,Traub_Ostro,Jarrat,Inverse_Jarrat))
Methods_st = ['Nw','NwM','CaWh','DcaWh','Ha','Ch','CaN/sH','Stir','Steff','Mid','Tr-Os','Ja','IfJa']
Parameters = ['Ord','Eff','NC','I/P','T','P/S','I/S'] #headers de la tablas
Numeric_Parameters = ['NC','I/P','T','P/S','I/S']
Conv_order = np.array([2,2,2,3,3,3,3,2,2,3,4,4,4])#orden de convergencia de los metodos
num_data = np.array([2,3,3,3,3,3,3,2,2,3,3,3,3])
Eff_coeff = Conv_order**(1/num_data) #indice \alpha^{1/d} de efeciencia computacional de los metodos

In [30]:
# Corremos los tiempos base para obetener los parametros de referencia.
np.set_printoptions(precision=8,suppress=True)
Newton_results1 = parameters_values_method(Newton_classicroot,xRb,xRb,func1,Dfunc1,roots,tol,maxite)
Newton_results2 = parameters_values_method(Newton_classicroot,xRs,yRs,func1,Dfunc1,roots,tol,maxite)
Newton_results3 = parameters_values_method(Newton_classicroot,xRb,xRb,func2,Dfunc2,roots,tol,maxite)
Newton_results4 = parameters_values_method(Newton_classicroot,xRs,yRs,func2,Dfunc2,roots,tol,maxite)

 59%|█████▉    | 620294/1048576 [06:24<04:25, 1612.70it/s]


KeyboardInterrupt: ignored

In [26]:
print(np.around([Newton_results1,Newton_results2,Newton_results3, Newton_results4], decimals=6))
#print(Newton_results1)
# todos los valores son importantes como referencia

[[     0.000015      7.598726      9.499314 110384.391157 838780.773169]
 [     0.            3.40537       4.787073 219043.227066 745923.18128 ]
 [     0.03034       7.785263    238.921868   4388.782029  34167.822617]
 [     0.            3.395221    108.222999   9689.031063  32896.399377]]


In [27]:
#Tabla 1
data = {
    'Ord': Conv_order,
    'Eff': Eff_coeff
}

index_labels = Methods_st # ['Nw','NwM','CaWh','DcaWh','Ha','Ch','CaN/sH','Stir','Steff','Mid','Tr-Os','Ja','IfJa']
column_labels = Parameters # ['Ord','Eff','NC','1/P','T','P/S','1/S']
ones = np.ones(3)
df = pd.DataFrame(data, index=index_labels, columns=column_labels)
# Convertimos los valores a tipos numéricos
df = df.astype(float)
# Colocamos los datos obtenidos
df.iloc[0, [2,3]] = Newton_results1[0:2]
df.iloc[0, 4:] = ones
for i,method in enumerate(Methods):
    Results = parameters_values_method(method,xRb,xRb,func1,Dfunc1,roots,tol,maxite)
    df.iloc[i+1, [2,3]] = Results[0:2]
    df.iloc[i+1, 4:] = Results[2:]/Newton_results1[2:]
# Imprimimos la tabla 1
print(df.round(6))
#Pasamos la tabla a formato latex
tabla_latex = df.to_latex(index=True)
print(tabla_latex)

100%|██████████| 1048576/1048576 [00:21<00:00, 49280.97it/s]


KeyboardInterrupt: ignored

In [ ]:
#Tabla 2
data = {
    'Ord': Conv_order,
    'Eff': Eff_coeff
}

index_labels = Methods_st # ['Nw','NwM','CaWh','DcaWh','Ha','Ch','CaN/sH','Stir','Steff','Mid','Tr-Os','Ja','IfJa']
column_labels = Parameters # ['Ord','Eff','NC','1/P','T','P/S','1/S']
ones = np.ones(3)
df = pd.DataFrame(data, index=index_labels, columns=column_labels)
# Convertimos los valores a tipos numéricos
df = df.astype(float)
# Colocamos los datos obtenidos
df.iloc[0, [2,3]] = Newton_results2[0:2]
df.iloc[0, 4:] = ones
for i,method in enumerate(Methods):
    Results = parameters_values_method(method,xRs,yRs,func1,Dfunc1,roots,tol,maxite)
    df.iloc[i+1, [2,3]] = Results[0:2]
    df.iloc[i+1, 4:] = Results[2:]/Newton_results2[2:]
# Imprimimos la tabla 1
print(df.round(6))
#Pasamos la tabla a formato latex
tabla_latex = df.to_latex(index=True)
print(tabla_latex)

In [ ]:
#Tabla 3
data = {
    'Ord': Conv_order,
    'Eff': Eff_coeff
}

index_labels = Methods_st # ['Nw','NwM','CaWh','DcaWh','Ha','Ch','CaN/sH','Stir','Steff','Mid','Tr-Os','Ja','IfJa']
column_labels = Parameters # ['Ord','Eff','NC','1/P','T','P/S','1/S']
ones = np.ones(3)
df = pd.DataFrame(data, index=index_labels, columns=column_labels)
# Convertimos los valores a tipos numéricos
df = df.astype(float)
# Colocamos los datos obtenidos
df.iloc[0, [2,3]] = Newton_results3[0:2]
df.iloc[0, 4:] = ones
for i,method in enumerate(Methods):
    Results = parameters_values_method(method,xRb,xRb,func2,Dfunc2,roots,tol,maxite)
    df.iloc[i+1, [2,3]] = Results[0:2]
    df.iloc[i+1, 4:] = Results[2:]/Newton_results3[2:]
# Imprimimos la tabla 1
print(df.round(6))
#Pasamos la tabla a formato latex
tabla_latex = df.to_latex(index=True)
print(tabla_latex)

In [ ]:
#Tabla 4
data = {
    'Ord': Conv_order,
    'Eff': Eff_coeff
}

index_labels = Methods_st # ['Nw','NwM','CaWh','DcaWh','Ha','Ch','CaN/sH','Stir','Steff','Mid','Tr-Os','Ja','IfJa']
column_labels = Parameters # ['Ord','Eff','NC','1/P','T','P/S','1/S']
ones = np.ones(3)
df = pd.DataFrame(data, index=index_labels, columns=column_labels)
# Convertimos los valores a tipos numéricos
df = df.astype(float)
# Colocamos los datos obtenidos
df.iloc[0, [2,3]] = Newton_results4[0:2]
df.iloc[0, 4:] = ones
for i,method in enumerate(Methods):
    Results = parameters_values_method(method,xRs,yRs,func2,Dfunc2,roots,tol,maxite)
    df.iloc[i+1, [2,3]] = Results[0:2]
    df.iloc[i+1, 4:] = Results[2:]/Newton_results4[2:]
# Imprimimos la tabla 1
print(df.round(6))
#Pasamos la tabla a formato latex
tabla_latex = df.to_latex(index=True)
print(tabla_latex)